# Understanding how the SQuAD dataset is set up for our task with BERT
  
We are going to fine-tune BERT for the text-extraction task with a dataset of questions and answers. The question are about a give paragraph (*context*) that contains the answers. The model will be trained to locate the answer in the context by giving the possitions where the answer starts and finishes.

In this notebook see how the dataset is set up for training.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

 More info:
  * [The Stanford Question Answering Dataset](https://rajpurkar.github.io/SQuAD-explorer/)
  * [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [1]:
import os
import json
import dataset_utils as du
from tensorflow import keras
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

## 1. The tokenizer

We are going to use the [Huggingface's tokenizers](https://huggingface.co/transformers/main_classes/tokenizer.html).

In [2]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",
                                               cache_dir=f"/scratch/snx3000/stud50/_bert_tockenizer")

save_path = f"/scratch/snx3000/stud50/bert_tockenizer"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(f"{save_path}/vocab.txt", lowercase=True)

## 2. The data

In [3]:
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url, cache_dir="./")
eval_path = keras.utils.get_file("eval.json", eval_data_url, cache_dir="./")

In [4]:
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

print(f"{len(raw_train_data['data'])} training items loaded.")
print(f"{len(raw_eval_data['data'])} evaluation items loaded.")

442 training items loaded.
48 evaluation items loaded.


In [5]:
raw_train_data.keys()

dict_keys(['data', 'version'])

In [6]:
raw_train_data['data'][91]['title']

'Alps'

In [7]:
raw_train_data['data'][91]['paragraphs'][0]['context']

'The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".'

In [8]:
raw_train_data['data'][91]['paragraphs'][0]['qas']

[{'answers': [{'answer_start': 190, 'text': 'Europe'}],
  'question': 'What Country are the Alps located in?',
  'id': '56f81f0ea6d7ea1400e173d7'},
 {'answers': [{'answer_start': 223, 'text': '1,200 kilometres'}],
  'question': 'How many kilometres do the Alps stretch?',
  'id': '56f81f0ea6d7ea1400e173d8'},
 {'answers': [{'answer_start': 475, 'text': 'over tens of millions of years'}],
  'question': 'How long has it taken for the Alps to form? ',
  'id': '56f81f0ea6d7ea1400e173d9'},
 {'answers': [{'answer_start': 732, 'text': 'Mont Blanc'}],
  'question': 'What is the highest mountain in the Alps?',
  'id': '56f81f0ea6d7ea1400e173da'},
 {'answers': [{'answer_start': 936, 'text': 'the "four-thousanders"'}],
  'question': 'The Alpine region is also known as what? ',
  'id': '56f81f0ea6d7ea1400e173db'}]

## 3.  The training set

In [9]:
%%time

max_len = 384

train_squad_examples = du.create_squad_examples(raw_train_data, max_len, tokenizer)
x_train, y_train = du.create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

86136 training points created.
CPU times: user 49.1 s, sys: 1.18 s, total: 50.3 s
Wall time: 50.3 s


In [10]:
len(x_train)

3

In [11]:
x_train[0].shape, x_train[1].shape, x_train[2].shape

((86136, 384), (86136, 384), (86136, 384))

In [12]:
sample = 20299

x_train[0][sample]

array([  101,  1996, 13698,  1006,  1013,  1097, 14277,  2015,  1013,
        1025,  3059,  1024,  2632,  8197,  1031,  1149,  2389,  8197,
        1033,  1025,  2413,  1024,  2632, 10374,  1031,  2632,  2361,
        1033,  1025,  2446,  1024,  2632, 11837,  1031,  1149, 29705,
        2389,  9737,  1033,  1025, 18326,  1024,  2632,  5051,  1031,
        1149,  2050, 23432, 14277, 29275,  1033,  1007,  2024,  1996,
        3284,  1998,  2087,  4866,  3137,  2846,  2291,  2008,  3658,
        4498,  1999,  2885,  1010, 10917,  3155,  1015,  1010,  3263,
        3717,  1006,  9683,  2771,  1007,  2408,  2809, 10348,  3032,
        1024,  5118,  1010,  2605,  1010,  2762,  1010,  3304,  1010,
       26500,  1010, 14497,  1010, 10307,  1010,  1998,  5288,  1012,
        1996, 16512,  4020,  2024,  3020,  1010,  1998,  1996, 24471,
        9777,  2936,  1010,  2021,  2119,  4682,  6576,  1999,  4021,
        1012,  1996,  4020,  2020,  2719,  2058, 15295,  1997,  8817,
        1997,  2086,

In [13]:
# The training sample is the text plus the question
#
# The padding zeroes are discarded by the tokenizer's decoding
tokenizer.decode(x_train[0][sample])

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

In [14]:
# `x_train[1][i]` is one for the elements that represent the question:
x_train[1][sample]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
# `x_train[1][i]==0` selects the context and `x_train[1][i]==1`, the question:
question_mask = (x_train[1][sample] == 1)
tokenizer.decode(x_train[0][sample][question_mask])

'how long has it taken for the alps to form?'

In [16]:
# `x_train[1][0]` is one for the elements that represent the text:
x_train[2][sample]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
# `x_train[2][i]==1` selects the part of the array that has text.
# The rest `x_train[2][i]==0` corresponds to zeros for padding
context_mask = (x_train[2][sample] == 1)
tokenizer.decode(x_train[0][sample][context_mask])

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

In [18]:
len(y_train)

2

In [19]:
# `y_train[0]` and `y_train[1]` are the possitions in the text where the answer starts and ends, respectively.
y_train[0].shape, y_train[0].shape

((86136,), (86136,))

A training sample looks like this:

In [20]:
print('\n * CONTEXT:                   \n', train_squad_examples[sample].context)
print('\n * QUESTION:                  \n', train_squad_examples[sample].question)
print('\n * ANSWER (REFERENCE):        \n', train_squad_examples[sample].answer_text)
print('\n * ANSWER IN CONTEXT:         \n', tokenizer.decode(x_train[0][sample][ y_train[0][sample]:y_train[1][sample]+1 ]))

print('\n\n === TRAINING SAMPLE ===')
print('\n * CONTEXT & QUESTION:        \n', tokenizer.decode(x_train[0][sample]))
print('\n * POSITION IN CONTEXT:       \n', (y_train[0][sample], y_train[1][sample]))


 * CONTEXT:                   
 The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".

 * QUE